In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, pipeline

In [ ]:
# tokenizer = LlamaTokenizer.from_pretrained("TheBloke/vicuna-13B-1.1-HF")
# model = AutoModelForCausalLM.from_pretrained("TheBloke/vicuna-13B-1.1-HF", device_map="auto", torch_dtype=torch.float16).eval()

# tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-65b-hf")
# model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-65b-hf", device_map="auto", torch_dtype=torch.float16).eval()

# tokenizer = AutoTokenizer.from_pretrained("mosaicml/mpt-7b-instruct")
# model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-7b-instruct", device_map="auto", torch_dtype=torch.float16, trust_remote_code=True).eval()

# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
# model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b", device_map="auto", torch_dtype=torch.float16, trust_remote_code=True).eval()

# tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5")
# model = AutoModelForCausalLM.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5", device_map="auto", torch_dtype=torch.float16).eval()

In [15]:
ag_news = load_dataset("ag_news", split="test")
ag_news[-1]

Found cached dataset ag_news (/home/kyle/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


{'text': 'EBay gets into rentals EBay plans to buy the apartment and home rental service Rent.com for \\$415 million, adding to its already exhaustive breadth of offerings.',
 'label': 2}

In [ ]:
# ag_news_frame = ag_news.to_pandas()
# labels = sorted(ag_news_frame["label"].unique().tolist())
# sample = pd.concat([ag_news_frame[ag_news_frame["label"] == label].sample(4) for label in labels])
# sample["text"].values.tolist()

In [ ]:
# current_summary = f"{ag_news[0]['text']}"

# task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

# Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
# Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

# Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
# Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

# Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
# Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

# Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
# Tweet: Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

# Now rewrite the following news summary in the style of a social media tweet.

# Summary: {current_summary}
# Tweet:"""

# input_prompt = f"User: {task_prompt}Assistant:".replace("</s>", " ").replace("<s>", " ")
# tokenized_prompt = tokenizer.encode(input_prompt, return_tensors="pt").to(model.device)
# with torch.no_grad():
#     outputs = model.generate(
#         tokenized_prompt,
#         max_new_tokens=500,
#         length_penalty=0,
#         early_stopping=True,
#         return_dict_in_generate=True,
#         output_scores=True,
#     )
    
# generation = tokenizer.decode(outputs["sequences"][0])
# print(generation)

# # decode the scores


In [ ]:
records = []

for index in tqdm(range(500)):
    current_entry = ag_news[index]
    current_summary = current_entry['text']
    task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
Tweet: WellPoint reports impressive Q3 growth! The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
Tweet: 💻 Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

Summary: {current_summary}
Tweet: """

    # input_prompt = f"User: {task_prompt}Assistant:".replace("</s>", " ").replace("<s>", " ")
    input_prompt = task_prompt
    tokenized_prompt = tokenizer.encode(input_prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            tokenized_prompt,
            max_new_tokens=50,
            length_penalty=0,
            do_sample=False,
            early_stopping=True,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generation = tokenizer.decode(outputs["sequences"][0]).split("Tweet:")[-1].split("</s>")[0].strip()
    records.append({
        "index": index,
        "original_summary": current_summary,
        "generated_summary": generation,
        "label": current_entry["label"]
    })
    
    print(f"\nOriginal Summary: {current_summary}")
    print(f"Generated Summary: {generation}")
        
    
records_frame = pd.DataFrame(records)
records_frame.to_csv("/home/kyle/repos/Parameter-Free-LM-Editing/datasets/ag_news_twitter/shifted_test_set_small_vicunna.csv", index=False)
records_frame

### MPT-7b

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:

for index in tqdm(range(200)):
    current_summary = ag_news[index]['text']
    task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
Tweet: Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

Now rewrite the following news summary in the style of a social media tweet.

Summary: {current_summary}
Tweet:"""

    generation = pipeline(
        task_prompt,
        max_new_tokens=200,
        do_sample=False,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    tweet = generation.split("Tweet:")[-1].split("\n")[0].strip()
    print("\nSummary:", current_summary)
    print(f"Tweet: {tweet}")

In [ ]:

# for index in range(10):
#     current_summary = ag_news[index]['text']
#     task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

# Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
# Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

# Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
# Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

# Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
# Tweet: 💻 Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

# Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
# Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

# Now rewrite the following news summary in the style of a social media tweet.

# Summary: {current_summary}
# Tweet: """

#     input_prompt = f"<|prompter|>{task_prompt}<|endoftext|><|assistant|>".replace("</s>", " ").replace("<s>", " ")
#     tokenized_prompt = tokenizer.encode(task_prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model.generate(
#             tokenized_prompt,
#             max_new_tokens=500,
#             length_penalty=0,
#             early_stopping=True,
#             return_dict_in_generate=True,
#             output_scores=True,
#         )

#     generation = tokenizer.decode(outputs["sequences"][0]).split("Tweet: ")[-1].split("</s>")[0].strip()
#     print(generation)

## Error Analysis

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("TheBloke/vicuna-7B-1.1-HF")
model = AutoModelForCausalLM.from_pretrained("TheBloke/vicuna-7B-1.1-HF", device_map="auto", torch_dtype=torch.float16).eval()

In [ ]:
inference_logs = pd.read_csv("/home/kyle/repos/Parameter-Free-LM-Editing/results/edit_experiment_2023-06-08_01-44-22/mosesju-distilbert-base-uncased-finetuned-news-ag_news_twitter-combined-inference-logs.csv")
inference_logs = inference_logs[["original input", "label", "original judgment", "TheBloke-vicuna-7B-1.1-HF-8 Judgment", "TheBloke-vicuna-7B-1.1-HF-8 Prompt", "TheBloke-vicuna-7B-1.1-HF-8 Input"]]
inference_logs
new_mistakes = inference_logs[(inference_logs["original judgment"] != inference_logs["TheBloke-vicuna-7B-1.1-HF-8 Judgment"]) & (inference_logs["original judgment"] == inference_logs["label"])]
new_mistakes

In [ ]:
print(new_mistakes.iloc[0]["original input"])
print("----")
print(new_mistakes.iloc[0]["TheBloke-vicuna-7B-1.1-HF-8 Prompt"])

In [ ]:
test_prompt = """User: Paraphrase the input text into the exact writing style of the following examples while keeping the same semantic meaning. Keep all facts and information.
Examples:
"Oracle Sets Its Sights on Small Biz LONDON -- Enterprise software giant Oracle may be known for big plans and bold moves, but the Redwood Shores, California, company has realized it also has to think small."
"SEC subpeonas Delphi over EDS payments Automotive parts supplier Delphi Corp. (NYSE: DPH) received a subpoena from the US Securities and Exchange Commission in late July in connection with agreements with Electronic Data Systems Corp."
"Delta trying to keep flying Delta Air Lines is cutting pay, increasing employees #39; share of health-care coverage and trimming retirement benefits - all to stave off bankruptcy."
"Why You Hate, and Like, Social Security Is Social Security an important part of civilized society, or just another form of welfare?"
"Turkish Leader Wins EU Officials' Backing (AP) AP - Turkish Prime Minister Recep Tayyip Erdogan's drive to get his country into the European Union won strong backing from top EU officials Friday, a week before the bloc's leaders decide whether to open membership talks."
"American cyclist keeps gold medal thanks to mistake from lab Helped by a laboratory #39;s blunder, Tyler Hamilton will be allowed to keep his Olympic cycling gold medal. The International Olympic Committee dropped its investigation Thursday into a blood test from the Summer"
"Sidebar: HP's grid storage initiative leads the way Hewlett-Packard was the first company to offer a grid storage device, but others are moving to catch up."
"Head of MI5 warns business to guard against terrorism The head of Britain #39;s security services has called on business to play a greater role in combating the threat of terrorism. Eliza Manningham-Buller, the"

Now paraphrase the current input text into the same style as the examples. Only return the paraphrased text for the below input text. MAke sure to keep all facts, information, and meaning.

Input Text: "🏀 Pacers bring back Jeff Foster from the injured list! 💪 The center is now active and ready to play for the Pacers. #PacersNation #NBA #JeffFoster"
Assistant:"""
# test_prompt = new_mistakes.iloc[0]["TheBloke-vicuna-7B-1.1-HF-8 Prompt"]

tokenized_prompt = tokenizer.encode(test_prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(
        tokenized_prompt,
        max_new_tokens=300,
        length_penalty=0,
        repetition_penalty=1.0,
        do_sample=True,
        temperature=0.7,
        early_stopping=True,
        return_dict_in_generate=True,
    )

    generation = tokenizer.decode(outputs["sequences"][0][len(tokenized_prompt[0]):]).replace("\n", " ").replace("</s>", "").strip()
    if "###" in generation:
        generation = generation.split("###")[0]
    if " Text:" in generation:
        generation = generation.split(" Text:")[1].strip()
    if "</s>" in generation:
        generation = generation.split("</s>")[0]
    if "<s>" in generation:
        generation = generation.replace("<s>", " ").strip()
    if generation.startswith('"') and generation.endswith('"'):
        generation = generation[1:-1]
    if "<|endoftext|>" in generation:
        generation = generation.split("<|endoftext|>")[0]
    if generation.startswith('"') and generation.endswith('"'):
        generation = generation[1:-1]

print(tokenizer.decode(outputs["sequences"][0]))

## GPT 3.5

In [ ]:
import os
import openai
from time import sleep
openai_api_key = ""
openai.api_key = openai_api_key

In [ ]:


messages = [
  {"role": "system", "content" : "Write the following news summary in the style of a Twitter post. Maintain all the relevant information and sentiment. Occasionally add emojis and a snarky tone."},
  {"role": "user", "content" : "The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket."},
]

completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages )
print(completion["choices"][0]["message"]["content"])

In [ ]:
range(len(pd.read_csv("ag_news_gpt3_responses_part1.csv")) - 24 , len(ag_news))


In [ ]:
def send_inference(news_summary):
    try:
        messages = [
            {"role": "system", "content" : "Write the following news summary in the style of a Twitter/social media. Maintain all the relevant information and sentiment."},
            {"role": "user", "content" : news_summary},
        ]
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, stop=["\n"])
        gpt_response = completion["choices"][0]["message"]["content"]
        return gpt_response
    except Exception as error:
        print(error)
        sleep(60)
        return send_inference(news_summary)


responses = []
for i in tqdm(range(len(pd.read_csv("ag_news_gpt3_responses_part1.csv")) - 24 , len(ag_news))):
    current_entry = ag_news[i]
    news_summary = current_entry["text"]
    gpt_response = send_inference(news_summary)
    
    current_entry["tweet"] = gpt_response
    responses.append(current_entry)
    pd.DataFrame(responses).to_csv("ag_news_gpt3_responses.csv", index=False)
    
    # print(f"\nNews: {news_summary}")
    # print(f"Tweet: {gpt_response}\n")

    

### Format AG Tweet

In [ ]:
def send_snarky_inference(news_summary):
    try:
        messages = [
            {"role": "system", "content" : "Write the following news summary in the style of a Twitter/social media. Maintain all the relevant information and sentiment. Add some flare with humor, anger, or sarcasm."},
            {"role": "user", "content" : news_summary},
        ]
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, top_p=0.9, frequency_penalty=0.5, presence_penalty=0.5)
        gpt_response = completion["choices"][0]["message"]["content"]
        return gpt_response.replace("\n", " ").replace("</s>", "").strip()
    except Exception as error:
        print(error)
        sleep(60)
        return send_snarky_inference(news_summary)


gpt_tweets = pd.read_csv("/home/kyle/repos/Parameter-Free-LM-Editing/notebooks/ag_news_gpt3_responses_part1.csv")
too_short_tweets = gpt_tweets[gpt_tweets["tweet"].str.len() < 40]
too_short_tweets.iloc[0]["tweet"]
responses = []
for i in tqdm(range(len(too_short_tweets))):
    current_entry = too_short_tweets.iloc[i]
    news_summary = current_entry["text"]
    gpt_response = send_snarky_inference(news_summary)
    
    current_entry["tweet"] = gpt_response
    responses.append(current_entry)
    pd.DataFrame(responses).to_csv("ag_news_gpt3_responses_snarky.csv", index=False)
    
    print(f"\nNews: {news_summary}")
    print(f"Tweet: {gpt_response}\n")
    

### Combine datasets

In [11]:
v1_prompt_tweets = pd.read_csv("/home/kyle/repos/Parameter-Free-LM-Editing/notebooks/ag_news_gpt3_responses_part1.csv")
v2_prompt_tweets = pd.read_csv("/home/kyle/repos/Parameter-Free-LM-Editing/notebooks/ag_news_gpt3_responses_part2.csv")
v1_filtered = v1_prompt_tweets[~v1_prompt_tweets["text"].isin(v2_prompt_tweets["text"])]
v1_filtered["prompt"] = "V1"
v2_prompt_tweets["prompt"] = "V2"
v1_filtered

/tmp/ipykernel_3535816/3022645870.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v1_filtered["prompt"] = "V1"


,text,label,tweet,prompt
0,Fears for T N pension after talks Unions repre...,2,Unions express disappointment in talks with Fe...,V1
1,The Race is On: Second Private Team Sets Launc...,3,🚀👨‍🚀 #SpaceRace update: Second private team to...,V1
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,"""Chemistry researcher's Ky. startup wins grant...",V1
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",V1
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,"""California takes on smog w emissions rules fo...",V1
...,...,...,...,...
7619,Around the world Ukrainian presidential candid...,0,BREAKING: Ukrainian presidential candidate Vik...,V1
7620,Void is filled with Clement With the supply of...,1,🚨RED SOX SIGN MATT CLEMENT!🚨 With pitching opt...,V1
7621,Martinez leaves bitter Like Roger Clemens did ...,1,"""Red Sox pitcher Pedro Martinez leaves team wi...",V1
7622,5 of arthritis patients in Singapore take Bext...,2,🚨 WARNING: Bextra and Celebrex painkillers may...,V1


In [13]:
combined_tweets_set = pd.concat([v1_filtered, v2_prompt_tweets]).drop_duplicates(subset=["text"])
combined_tweets_set

,text,label,tweet,prompt
0,Fears for T N pension after talks Unions repre...,2,Unions express disappointment in talks with Fe...,V1
1,The Race is On: Second Private Team Sets Launc...,3,🚀👨‍🚀 #SpaceRace update: Second private team to...,V1
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,"""Chemistry researcher's Ky. startup wins grant...",V1
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",V1
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,"""California takes on smog w emissions rules fo...",V1
...,...,...,...,...
511,"2005 American Bowl teams announced New York, N...",1,🚨BREAKING NEWS🚨: The 2005 American Bowl teams ...,V2
512,"NBA Game Summary - Denver at Miami Miami, FL (...",1,🏀 Miami Heat dominate Denver Nuggets 107-100 l...,V2
513,Bosnian-Serb prime minister resigns in protest...,0,🚨BREAKING🚨 Bosnian-Serb Prime Minister throws ...,V2
514,Mortaza strikes to lead superb Bangladesh rall...,1,🚨BREAKING NEWS🚨 Mortaza's lethal bowling skill...,V2


In [26]:
joined_frame = ag_news.to_pandas().join(combined_tweets_set.set_index("text"), on="text", lsuffix="_ag_news", rsuffix="_gpt3")
joined_frame.rename(columns={"text": "article summary", "tweet": "tweet summary", "label_ag_news": "label"}, inplace=True)
joined_frame.drop(columns=["label_gpt3"], inplace=True)
joined_frame

,article summary,label,tweet summary,prompt
0,Fears for T N pension after talks Unions repre...,2,Unions express disappointment in talks with Fe...,V1
1,The Race is On: Second Private Team Sets Launc...,3,🚀👨‍🚀 #SpaceRace update: Second private team to...,V1
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,"""Chemistry researcher's Ky. startup wins grant...",V1
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",V1
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,"""California takes on smog w emissions rules fo...",V1
...,...,...,...,...
7595,Around the world Ukrainian presidential candid...,0,BREAKING: Ukrainian presidential candidate Vik...,V1
7596,Void is filled with Clement With the supply of...,1,🚨RED SOX SIGN MATT CLEMENT!🚨 With pitching opt...,V1
7597,Martinez leaves bitter Like Roger Clemens did ...,1,"""Red Sox pitcher Pedro Martinez leaves team wi...",V1
7598,5 of arthritis patients in Singapore take Bext...,2,🚨 WARNING: Bextra and Celebrex painkillers may...,V1


In [35]:
joined_frame[joined_frame["tweet summary"].str.startswith("🚨BREAKING NEWS🚨")]
joined_frame["tweet summary"] = joined_frame["tweet summary"].str.replace("🚨BREAKING NEWS🚨", "")
joined_frame.to_csv("/home/kyle/repos/Parameter-Free-LM-Editing/datasets/ag_news_twitter/shifted_test_set_gpt3.csv", index=False)
joined_frame

,article summary,label,tweet summary,prompt
0,Fears for T N pension after talks Unions repre...,2,Unions express disappointment in talks with Fe...,V1
1,The Race is On: Second Private Team Sets Launc...,3,🚀👨‍🚀 #SpaceRace update: Second private team to...,V1
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,"""Chemistry researcher's Ky. startup wins grant...",V1
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",V1
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,"""California takes on smog w emissions rules fo...",V1
...,...,...,...,...
7595,Around the world Ukrainian presidential candid...,0,BREAKING: Ukrainian presidential candidate Vik...,V1
7596,Void is filled with Clement With the supply of...,1,🚨RED SOX SIGN MATT CLEMENT!🚨 With pitching opt...,V1
7597,Martinez leaves bitter Like Roger Clemens did ...,1,"""Red Sox pitcher Pedro Martinez leaves team wi...",V1
7598,5 of arthritis patients in Singapore take Bext...,2,🚨 WARNING: Bextra and Celebrex painkillers may...,V1
